# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [231]:
def priest(Sex,Age,Rrate,Spo2,Hrate,Bpressure,Temperature,Alertness,InspiredOxygen,Performance):
    """
    The priest function computes(gets the score from each of the input argument) and returns the risk percentage by considering input factors.

    Parameters:
      Sex (stirng) : gender of person(one of male or female(case insensitive))
      Age (integer) : age of the person in years
      Rrate (float) : Respiratory rate in breaths per minute
      Spo2 (float) : Oxygen saturation as a percent between 0 and 1
      Hrate (float) : Heart rate in beats per minute
      Bpressure (float) : Systolic BP in mmHg
      Temperature (float) : Temperature in degrees C
      Alertness (string) : Alertness as a string description
      Inspired_oxygen (string) : Inspired Oxygen as as string description
      Performance (string) : Performance Status as a string description

    Returns:
      Float: Returns the risk percentage derived from the MDCalc website based on the input parameters
      Raises ValueError with the invalid argument whenever the input is not valid

    >>> priest("male",40,23.0,93,105.0,115.0,38.1,"ALERT","supplemental oxygen","unrestricted normal activity")
    0.26
    >>> priest("FEMALE",40, 8.0,96,105.0,115.0,35.4,"ALERT","AIR","unrestricted normal activity")
    0.09
    >>> priest("female",66,12.0,98, 41.0, 115.0, 34.9, "ALERT", "air", "unrestricted normal activity")
    0.18
    >>> priest("male",18,24.0,99,105.0,115.0,38.1,"ALERT","AIR","unrestricted normal activity")
    0.09
    """


    score=0

    #SEX
    Sex=Sex.upper()
    if Sex=="MALE": score+=1;
    elif Sex =="FEMALE": score += 0
    else: raise ValueError("Input Sex is not valid")

    #Age
    if Age<50:
        score+=0
    elif Age>=50 and Age<=65:
        score+=2
    elif Age>=66 and Age<=80:
        score+=3
    else:
        score+=4

    #respiratory rate
    if Rrate<9:
        score+=3
    elif Rrate>=9 and Rrate<=11:
        score+=1
    elif Rrate>=12 and Rrate<=20:
        score+=0
    elif Rrate>=21 and Rrate<=24:
        score+=2
    else:
        score+=3

    #Oxygen saturation
    if Spo2<=95 and Spo2>=94:
        score+=1
    elif Spo2<=93 and Spo2>=92:
        score+=2
    elif Spo2<92:
        score+=3

    #Heart rate
    if Hrate<41:
        score+=3
    elif Hrate>=41 and Hrate<=50:
        score+=1
    elif Hrate>=51 and Hrate<=90:
        score+=0
    elif Hrate>=91 and Hrate<=110:
        score+=1
    elif Hrate>=111 and Hrate<=130:
        score+=2
    elif Hrate>130:
        score+=3

    #Blood pressure in mmHg
    if Bpressure<91:
        score+=3
    elif Bpressure>=91 and Bpressure<=100:
        score+=2
    elif Bpressure>=101 and Bpressure<=110:
        score+=1
    elif Bpressure>=111 and Bpressure<=219:
        score+=0
    else:
        score+=3

    if Temperature<35.1:
        score+=3
    elif Temperature>=35.1 and Temperature<=36.0:
        score+=1
    elif Temperature>=36.1 and Temperature<=38.0:
        score+=0
    elif Temperature>=38.1 and Temperature<=39.0:
      score+=1
    else:
      score+=2

    #Alertness
    Alertness=Alertness.upper()

    if Alertness=="ALERT":
        score+=0
    else:
        score+=3

    #Inspired Oxygen
    InspiredOxygen=InspiredOxygen.upper()
    if InspiredOxygen=="AIR":
        score+=0
    else:
        score+=2

    Performance=Performance.upper()

    if Performance=="Unrestricted normal activity".upper():
        score+=0
    elif Performance=="Limited strenuous activity, can do light activity".upper():
        score+=1

    elif Performance=="Limited activity, can self-care".upper():
        score+=2
    elif Performance=="Limited self-care".upper():
        score+=3
    elif Performance=="Bed/chair bound, no self-care".upper():
        score+=4

    conversion=[1, 1, 2, 2, 3, 9, 15, 18, 22, 26, 29, 34, 38, 46, 47,49,55]

    if score <= 16:
        return conversion[score]/100
    elif score >= 17 and score <= 25:
        return 59
    return 99

In [232]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("male",40,23.0,93,105.0,115.0,38.1,"ALERT","supplemental oxygen","unrestricted normal activity")
Expecting:
    0.26
ok
Trying:
    priest("FEMALE",40, 8.0,96,105.0,115.0,35.4,"ALERT","AIR","unrestricted normal activity")
Expecting:
    0.09
ok
Trying:
    priest("female",66,12.0,98, 41.0, 115.0, 34.9, "ALERT", "air", "unrestricted normal activity")
Expecting:
    0.18
ok
Trying:
    priest("male",18,24.0,99,105.0,115.0,38.1,"ALERT","AIR","unrestricted normal activity")
Expecting:
    0.09
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [237]:
import requests

def find_hospital(age, sex, risk):
    """
   this function retrieves the hospital name based on age, sex, and risk percentage.
    this function returns the name of the hospital if found, None otherwise.


    Examples:
        >>> find_hospital(40, 'male', 0.1)
        'Southwest Hospital and Medical Center'

        >>> find_hospital(20, 'female', 0.96)
        'Emory Dunwoody Medical Center'

        >>> find_hospital(72, 'male', 0.22)
        'Wesley Woods Geriatric Hospital'


    """
    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    response = requests.get(url)
    hospital_data = response.json()
    hospital_name = hospital_data.get('hospital')
    return hospital_name



In [238]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(20, 'female', 0.96)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(72, 'male', 0.22)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [239]:
import json

def get_address(hospital_name, file_path='/content/ga_hospitals.json'):
    """
    This function proivdes the address of the given hospital name from the loaded JSON data.
    This function returns the address of the hospital if found, None otherwise.
    Examples:
        >>> get_address('SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA')
        '1821 CLIFTON ROAD NE'

        >>> get_address('EMORY DUNWOODY MEDICAL CENTER')
        '4500 NORTH SHALLOWFORD ROAD'

        >>> get_address('LAUREL HEIGHTS HOSPITAL')
        '934 BRIARCLIFF ROAD'
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
        caps = hospital_name.upper()
        return data.get(caps, {}).get('ADDRESS', None)


In [240]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA')
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address('EMORY DUNWOODY MEDICAL CENTER')
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok
Trying:
    get_address('LAUREL HEIGHTS HOSPITAL')
Expecting:
    '934 BRIARCLIFF ROAD'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [241]:
def calculate_risk(patient_data):
    """
    The provided function calculates risk scores for patients based on age and BMI.
    The provided function gives a list of tuples containing patient IDs and corresponding risk scores.
    Examples:
        >>> patient_data = [{'Patient_ID': 'E5361', 'Age': '18', 'BMI': '22'}]
        >>> calculate_risk(patient_data)
        [('E9559', 960.0)]

        >>> patient_data = [{'Patient_ID': 'E7130', 'Age': '18', 'BMI': '22'}]
        >>> calculate_risk(patient_data)
        [('E9385', 969.0)]
    """
    risk_scores = []
    for patient in patient_data:
        # Extract relevant patient information
        age = int(patient['Age'])
        bmi = float(patient['BMI'])
        # Calculate risk based on your algorithm
        risk = age * bmi  # Replace this with your actual risk calculation
        risk_scores.append((patient['Patient_ID'], risk))

    return risk_scores



## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [242]:
import json
import gdown

# Load your calculated results
with open('people_results.json', 'r') as your_results_file:
    your_results = json.load(your_results_file)

# Define the Google Drive file URL for the expected results
expected_results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

# Download the expected results file from Google Drive
gdown.download(expected_results_url, 'expected_results.json', quiet=False)

# Load the expected results from the downloaded file
with open('expected_results.json', 'r') as expected_results_file:
    expected_results = json.load(expected_results_file)

# Compare your results with the expected results
if your_results == expected_results:
    print("Your results match the expected results.")
else:
    print("Your results do not match the expected results.")

Downloading...
From: https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5
To: /content/expected_results.json
100%|██████████| 29.4k/29.4k [00:00<00:00, 32.4MB/s]

Your results match the expected results.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---